In [1]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [2]:
""" pdf_loader = DirectoryLoader('../../training_data/', glob="**/*.pdf")
txt_loader = DirectoryLoader('../../training_data/', glob="**/*.txt")
word_loader = DirectoryLoader('../../training_data/', glob="**/*.docx")
csv_loader = DirectoryLoader('../../training_data/', glob="**/*.csv") """

' pdf_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.pdf")\ntxt_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.txt")\nword_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.docx")\ncsv_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.csv") '

In [3]:
from bs4 import BeautifulSoup as Soup

url = "https://developers.hubspot.com/docs/cms/hubl"
recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=210, extractor=lambda x: Soup(x, "html.parser").text)

loaders = [recursive_url_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Total number of documents: 34


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings()

Created a chunk of size 11806, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 2934, which is longer than the specified 1000
Created a chunk of size 1552, which is longer than the specified 1000
Created a chunk of size 1250, which is longer than the specified 1000
Created a chunk of size 1770, which is longer than the specified 1000
Created a chunk of size 1263, which is longer than the specified 1000
Created a chunk of size 1404, which is longer than the specified 1000
Created a chunk of size 3872, which is longer than the specified 1000
Created a chunk of size 4866, which is longer than the specified 1000
Created a chunk of size 1203, which is longer than the specified 1000
Created a chunk of size 1311, which is longer than the specified 1000
Created a chunk of size 3399, which is longer than the specified 1000
Created a chunk of 

In [6]:
db = Chroma.from_documents(texts, embeddings, collection_name="hubspot_hubl_knowledge_base")

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain.llms import Cohere
cohere_api_key= os.getenv('COHERE_API_KEY')
cohere_llm = Cohere(cohere_api_key=cohere_api_key)

ImportError: Could not import cohere python package. Please install it with `pip install cohere`.

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
model_path = ""
n_gpu_layers = 250  # Change this value based on your model and your GPU VRAM pool.
n_batch = 1024  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
# Make sure the model path is correct for your system!
llm2 = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=10240,
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../../../../models/llama2_variants/llama-2-7b-32k-instruct.Q4_K_S.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_down.weight q5_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    7:              blk.0.ffn_up.wei

In [ ]:
hubl_docs_chain = RetrievalQA.from_chain_type(
    llm=llm2, chain_type="stuff", retriever=db.as_retriever()
)

In [ ]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import DuckDuckGoSearchRun 
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools import ShellTool
from langchain.agents import load_tools

In [ ]:
#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()
shell_tool = ShellTool()


#Python REPL Class
python_repl = PythonREPL()

shell_tool = Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
)


#DuckDuckGo Tool
duckduck_tool=Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
)

#Python REPL Agent Tool Class
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)

In [ ]:
tools = [
    Tool(
        name="HubL Docs",
        func=hubl_docs_chain.run,
        description="Useful for when you need to answer questions about HubL. output should be code",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, cohere_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    "What is HubL?"
)



> Entering new AgentExecutor chain...
 I need to answer the question: "What is HubL?"
Action: HubL Docs|explain what HubL is
        Observation: HubL is a markup language based on JavaScript syntax used for creating modules in Hubspot CMS
Thought: I now know the final answer
Final Answer: HubL is a markup language based on JavaScript syntax used for creating modules in Hubspot CMS

> Finished chain.


'HubL is a markup language based on JavaScript syntax used for creating modules in Hubspot CMS'